In [40]:
import math
from textblob import TextBlob as tb
import newspaper
import os
import random
import pickle
import import_ipynb
from lexrank_summarizer import summarize
from news_classes import customizedPaper
from news_classes import paragraph

In [41]:
from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import render_template
from flask import request

In [42]:
cwd=os.getcwd()+'/'
news_path='news_30_9_2018.pkl'

with open(news_path, 'rb') as input:
    allnews = pickle.load(input)

In [43]:
def getSummarizedFromList(para_list):
    summ_list=[]
    for paragraph in para_list:
        obj={'title':paragraph.title, 'url':paragraph.url, 'text':summarize(paragraph.body, 5), 'date':paragraph.date, 'brand':paragraph.brand}
        summ_list.append(obj)    
    
    print(len(para_list))
    print(len(summ_list))
    
    return summ_list

In [44]:
def searchNews(searched_key):
    key_words=searched_key.split()
 
    para_list=[]
    count_list=[]
    
    for paragraph in allnews.paragraphs:
        matched_count=0
        for word in key_words:
            word=word.lower()
            if paragraph.title is not None and paragraph.body is not None:
                if word in paragraph.title.lower():
                    matched_count+=1
        if matched_count>0 and paragraph.date is not None:
            para_list.append(paragraph)
            count_list.append(matched_count)
    return para_list, count_list

In [45]:
def sortNews(para_list, count_list):
    para_list.sort(key=lambda item:(item.date.year, item.date.month, item.date.day), reverse=True)
    #para_list = [x for _, x in sorted(zip(count_list,para_list), reverse=True)]
    
    return para_list

In [46]:
def getNewsFromKey(searched_key):
    para_list, count_list = searchNews(searched_key)
    para_list = sortNews(para_list, count_list)

    return para_list

In [47]:
list_of_news=[]
searched_word=""
total_page=0
content_per_page=5

In [48]:
app = Flask(__name__)

In [49]:
@app.route('/show_result', methods=['POST'])
def show_result():
    global searched_word
    searched_word = request.form['news_keyword']
    print(searched_word)
    global list_of_news
    list_of_news=getNewsFromKey(searched_word)
        
    global total_page
    total_page=math.ceil(len(list_of_news)/content_per_page)
    
    list_ten=[]
    for i in range(min(len(list_of_news), content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    
    return rendered

In [50]:
@app.route('/show_other_page', methods=['POST'])
def show_other_page():
    page_number = int(request.form['page_number'])
        
    list_ten=[]
    for i in range(page_number*content_per_page, min(len(list_of_news), (page_number+1)*content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    
    return rendered

In [51]:
@app.route("/")
def search_key():
    rendered = render_template('ani_search.html')
    #rendered="hello there!"
    return rendered

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('0.0.0.0', 9000, app)

 * Running on http://0.0.0.0:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2018 00:05:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2018 00:05:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2018 00:05:48] "POST /show_result HTTP/1.1" 200 -


dhaka university
5
2
